In [ ]:
def zoom_image(image, zoom_factor=1.2):
    width, height = image.size
    new_width = int(width * zoom_factor)
    new_height = int(height * zoom_factor)
    zoomed_image = image.resize((new_width, new_height), Image.Resampling.LANCZOS)
    return zoomed_image.crop((0, 0, width, height))

def add_noise(image):
    image_array = np.array(image)
    row, col = image_array.shape
    gauss = np.random.normal(0, 25, (row, col))
    noisy_image = np.array(np.clip(image_array + gauss, 0, 255), dtype=np.uint8)
    return Image.fromarray(noisy_image)

def shear_image(image, shear_factor=0.2):
    width, height = image.size
    matrix = [1, shear_factor, 0, shear_factor, 1, 0]
    return image.transform((width, height), Image.AFFINE, matrix)

def random_crop(image, crop_size=(400, 400)):
    width, height = image.size
    crop_width, crop_height = crop_size
    x_offset = random.randint(0, width - crop_width)
    y_offset = random.randint(0, height - crop_height)
    return image.crop((x_offset, y_offset, x_offset + crop_width, y_offset + crop_height))

def perspective_transform(image):
    image_array = np.array(image)
    rows, cols = image_array.shape
    pts1 = np.float32([[50,50], [200,50], [50,200], [200,200]])
    pts2 = np.float32([[10,100], [200,50], [100,250], [210,250]])
    M = cv2.getPerspectiveTransform(pts1, pts2)
    return cv2.warpPerspective(image_array, M, (cols, rows))

def random_blur(image):
    return image.filter(ImageFilter.GaussianBlur(radius=random.uniform(0.1, 2.0)))

def adjust_brightness(image, factor=1.2):
    enhancer = ImageEnhance.Brightness(image)
    return enhancer.enhance(factor)

def augment_image(image):
    augmented_images = []
    augmented_images.append(image)
    augmented_images.append(ImageOps.mirror(image))
    augmented_images.append(ImageOps.flip(image))
    augmented_images.append(image.rotate(random.randint(-30, 30), expand=True))
    augmented_images.append(zoom_image(image))
    augmented_images.append(add_noise(image))
    augmented_images.append(shear_image(image))
    augmented_images.append(random_crop(image))
    augmented_images.append(random_blur(image))
    augmented_images.append(adjust_brightness(image, random.uniform(0.8, 1.2)))
    return augmented_images

In [ ]:
DIR = "Cropped"
save_dir = "Padding"

model_path = "Trainer_LBHF.yml"

recognizer = cv2.face.LBPHFaceRecognizer_create()
#recognizer = cv2.face.EigenFaceRecognizer_create()
#recognizer = cv2.face.FisherFaceRecognizer_create()

current_id = 0
label_ids = {}
Names = []
y_labels = []
x_train = []

desired_width = 500
desired_height = 550

for person_folder in os.listdir(DIR):
    person_path = os.path.join(DIR, person_folder)
    if os.path.isdir(person_path) and person_folder != 'default':
        for image_file in os.listdir(person_path):
            img_path = os.path.join(person_path, image_file)
            img_label = os.path.basename(person_path)

            if not img_label in label_ids:
                label_ids[img_label] = current_id
                current_id += 1
            id_ = label_ids[img_label]

            pill_img = Image.open(img_path).convert("L")

            width, height = pill_img.size
            if width >= desired_width and height >= desired_height:
                img_re = pill_img.resize((desired_width, desired_height), Image.Resampling.LANCZOS)
            else:
                top = (desired_height - height) // 2
                bottom = desired_height - height - top
                left = (desired_width - width) // 2
                right = desired_width - width - left

                img_re = Image.new('L', (desired_width, desired_height), (0))
                img_re.paste(pill_img, (left, top)) 

                img_re = img_re.resize((desired_width, desired_height), Image.Resampling.LANCZOS)
                
            img_re.save(os.path.join(save_dir, f"original_{person_folder}_{image_file}"))

            img_array = np.array(img_re, "uint8")
            x_train.append(img_array)
            y_labels.append(id_)
            
            augmented_images = augment_image(img_re)
            
            for aug_img in augmented_images:
                img_array = np.array(aug_img, "float32") / 255.0
                x_train.append(img_array)
                y_labels.append(id_)

x_train, y_labels = shuffle(x_train, y_labels, random_state=42)

with open("Labels_faces.pickle", 'wb') as f:
    pickle.dump(label_ids, f)

recognizer.train(x_train, np.array(y_labels))
recognizer.save(model_path)
if os.path.exists(model_path):
    print(f"Model successfully saved to {model_path}")
else:
    print(f"Failed to save the model to {model_path}")


In [ ]:
import os
import numpy as np
import dlib
from sklearn.svm import SVC
from PIL import Image
import pickle

DIR = "Cropped_50"

shape_predictor_path = "shape_predictor_68_face_landmarks.dat"
face_rec_model_path = "dlib_face_recognition_resnet_model_v1.dat"

detector = dlib.get_frontal_face_detector()
sp = dlib.shape_predictor(shape_predictor_path)
facerec = dlib.face_recognition_model_v1(face_rec_model_path)

label_ids = {}
current_id = 0
embeddings_list = []
labels = []

for person_folder in os.listdir(DIR):
    person_path = os.path.join(DIR, person_folder)
    if os.path.isdir(person_path) and person_folder != 'default':
        for image_file in os.listdir(person_path):
            img_path = os.path.join(person_path, image_file)
            img_label = os.path.basename(person_path)

            if img_label not in label_ids:
                label_ids[img_label] = current_id
                current_id += 1
            id_ = label_ids[img_label]

            img = Image.open(img_path).convert("RGB")
            img_cv = np.array(img)

            dets = detector(img_cv, 1)
            
            for k, d in enumerate(dets):
                shape = sp(img_cv, d)
                
                embedding = facerec.compute_face_descriptor(img_cv, shape)
                embedding = np.array(embedding)

                embeddings_list.append(embedding)
                labels.append(id_)

X = np.array(embeddings_list)
y = np.array(labels)

clf = SVC(kernel='linear')
clf.fit(X, y)

with open("face_recognition_model.pkl", "wb") as f:
    pickle.dump(clf, f)

print("Model trained and saved as face_recognition_model.pkl")
